# Import Library

In [55]:
import numpy as np
import pandas as pd
import tensorflow as tf
import faiss
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import classification_report

from tensorflow.keras import layers, models
from sentence_transformers import SentenceTransformer

# Prepare Data

In [30]:
df = pd.read_csv("../notebooks/anang/export/final_data.csv")
df.head()

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,company_process_time,company_size,company_industry,job_description,salary,job_description_cleaned,annotations,skills
0,1,Facility Maintenance & Smart Warehouse Manager,Bandung,IDR,Manajer/Asisten Manajer,5 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Manufaktur,Pemeliharaan",NaN,NaN,NaN,NaN,Deskripsi PekerjaanRequirements :D3/SI from re...,NaN,deskripsi pekerjaanrequirements si from reputa...,{'text': 'deskripsi pekerjaanrequirements si f...,"['electrical inspection', 'management system',..."
1,2,Procurement Department Head,Jakarta Raya,IDR,Manajer/Asisten Manajer,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Manufaktur,Pembelian/Manajemen Material",NaN,25 days,51 - 200 pekerja,Manajemen/Konsulting HR,Job Role: 1. Responsible for material availabi...,NaN,job role responsible for material availabili...,{'text': 'job role responsible for material av...,"['heavy equipment', 'contract management', 'pr..."
2,3,SALES ADMIN,Jakarta Barat,IDR,Supervisor/Koordinator,4 tahun,Sarjana (S1),Penuh Waktu,"Penjualan / Pemasaran,Penjualan Ritel","Waktu regular, Senin - Jumat;Bisnis (contoh: K...",30 days,51 - 200 pekerja,Umum & Grosir,Internal Sales & AdminJob Description :We are ...,NaN,internal sales adminjob description we are loo...,{'text': 'internal sales adminjob description ...,"['microsoft office', 'heat exchanger', 'carbon..."
3,4,City Operation Lead Shopee Express (Cirebon),Cirebon,IDR,Supervisor/Koordinator,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Pelayanan,Logistik/Rantai Pasokan","Tip;Waktu regular, Senin - Jumat;Kasual (conto...",21 days,2001 - 5000 pekerja,Retail/Merchandise,Job Description:Responsible for HSE implementa...,NaN,job description responsible for hse implementa...,{'text': 'job description responsible for hse ...,"['operation management', 'analytical skill', '..."
4,5,Japanese Interpreter,Bekasi,IDR,Pegawai (non-manajemen & non-supervisor),2 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Lainnya,Jurnalis/Editor",NaN,23 days,201 - 500 pekerja,Manajemen/Konsulting HR,Overview: Our clients is manufacture for autom...,NaN,overview our clients is manufacture for automo...,{'text': 'overview our clients is manufacture ...,"['japanese', 'translator', 'english', 'non', '..."


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17182 entries, 0 to 17181
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       17182 non-null  int64  
 1   job_title                17182 non-null  object 
 2   location                 17182 non-null  object 
 3   salary_currency          17178 non-null  object 
 4   career_level             17182 non-null  object 
 5   experience_level         17182 non-null  object 
 6   education_level          17182 non-null  object 
 7   employment_type          17182 non-null  object 
 8   job_function             17182 non-null  object 
 9   job_benefits             14440 non-null  object 
 10  company_process_time     13377 non-null  object 
 11  company_size             15488 non-null  object 
 12  company_industry         16412 non-null  object 
 13  job_description          17182 non-null  object 
 14  salary                

In [32]:
# Get a NumPy array of all unique values
unique_values_list = df['job_function'].unique()
print(len(unique_values_list))

68


In [33]:
job_function_mapping = {

    # Finance & Accounting
    "Akuntansi / Keuangan,Audit & Pajak": "Finance & Accounting",
    "Akuntansi / Keuangan,Akuntansi Umum / Pembiayaan": "Finance & Accounting",
    "Akuntansi / Keuangan,Perbankan / Jasa Finansial ": "Finance & Accounting",
    "Akuntansi / Keuangan,Keuangan / Investasi Perusahaan ": "Finance & Accounting",
    "Sains,Aktuaria/Statistik": "Finance & Accounting",

    # IT & Software
    "Komputer/Teknologi Informasi,IT-Perangkat Lunak": "IT & Software",
    "Komputer/Teknologi Informasi,IT-Admin Jaringan/Sistem/Database": "IT & Software",
    "Komputer/Teknologi Informasi,IT-Perangkat Keras": "IT & Software",

    # Engineering (Non-IT)
    "Teknik,Teknik Lingkungan": "Engineering",
    "Teknik,Teknik Lainnya": "Engineering",
    "Teknik,Mekanikal": "Engineering",
    "Teknik,Teknik Elektronika": "Engineering",
    "Teknik,Teknik Elektro": "Engineering",
    "Teknik,Teknik Industri": "Engineering",
    "Teknik,Teknik Kimia": "Engineering",
    "Bangunan/Konstruksi,Teknik Sipil/Konstruksi Bangunan": "Engineering",
    "Bangunan/Konstruksi,Survei Kuantitas": "Engineering",
    "Sains,Pertanian": "Engineering",
    "Sains,Teknologi Makanan/Ahli Gizi": "Engineering",
    "Sains,Sains & Teknologi": "Engineering",

    # Manufacturing & Operations
    "Manufaktur,Manufaktur": "Manufacturing & Operations",
    "Manufaktur,Pemeliharaan": "Manufacturing & Operations",
    "Manufaktur,Pembelian/Manajemen Material": "Manufacturing & Operations",
    "Manufaktur,Penjaminan Kualitas / QA": "Manufacturing & Operations",
    "Pelayanan,Logistik/Rantai Pasokan": "Manufacturing & Operations",

    # Sales & Marketing
    "Penjualan / Pemasaran,Pemasaran/Pengembangan Bisnis": "Sales & Marketing",
    "Penjualan / Pemasaran,Digital Marketing": "Sales & Marketing",
    "Penjualan / Pemasaran,Penjualan - Jasa Keuangan": "Sales & Marketing",
    "Penjualan / Pemasaran,Penjualan - Teknik/Teknikal/IT": "Sales & Marketing",
    "Penjualan / Pemasaran,Penjualan Ritel": "Sales & Marketing",
    "Penjualan / Pemasaran,Merchandising": "Sales & Marketing",
    "Penjualan / Pemasaran,Penjualan - Korporasi": "Sales & Marketing",
    "Penjualan / Pemasaran,Telesales/Telemarketing": "Sales & Marketing",
    "Penjualan / Pemasaran,E-commerce": "Sales & Marketing",
    "Seni/Media/Komunikasi,Periklanan": "Sales & Marketing",

    # HR & Administration
    "Sumber Daya Manusia/Personalia,Sumber Daya Manusia / HR": "HR & Administration",
    "Sumber Daya Manusia/Personalia,Staf / Administrasi umum": "HR & Administration",
    "Sumber Daya Manusia/Personalia,Sekretaris": "HR & Administration",
    "Sumber Daya Manusia/Personalia,Top Management / Manajemen Tingkat Atas": "HR & Administration",
    "Pelayanan,Pengacara / Asisten Legal": "HR & Administration",
    "Pelayanan,Angkatan Bersenjata": "HR & Administration",

    # Creative & Media
    "Seni/Media/Komunikasi,Seni / Desain Kreatif": "Creative & Media",
    "Seni/Media/Komunikasi,Hubungan Masyarakat": "Creative & Media",
    "Seni/Media/Komunikasi,Hiburan": "Creative & Media",
    "Lainnya,Jurnalis/Editor": "Creative & Media",

    # Education & Training
    "Pendidikan/Pelatihan,Pendidikan": "Education",
    "Pendidikan/Pelatihan,Pelatihan & Pengembangan": "Education",

    # Hospitality & Service
    "Hotel/Restoran,Makanan/Minuman/Pelayanan Restoran": "Hospitality & Service",
    "Hotel/Restoran,Hotel/Pariwisata": "Hospitality & Service",
    "Pelayanan,Layanan Pelanggan": "Hospitality & Service",
    "Pelayanan,Teknikal & Bantuan Pelanggan": "Hospitality & Service",
    "Pelayanan,Perawatan Pribadi": "Hospitality & Service",
    "Layanan Kesehatan,Diagnosa/Lainnya": "Hospitality & Service",
    "Layanan Kesehatan,Farmasi": "Hospitality & Service",
    "Layanan Kesehatan,Praktisi/Asisten Medis": "Hospitality & Service",
    "Sains,Bioteknologi": "Hospitality & Service",
    "Sains,Biomedis": "Hospitality & Service",
    "Sains,Kimia": "Hospitality & Service",
    "Sains,Geologi/Geofisika": "Hospitality & Service",

    # Other
    "Bangunan/Konstruksi,Arsitek/Desain Interior": "Other",
    "Bangunan/Konstruksi,Properti/Real Estate": "Other",
    "Lainnya,Pekerjaan Umum": "Other",
    "Lainnya,Lainnya/Kategori tidak tersedia": "Other",
}

df["job_function_group"] = (
    df["job_function"]
    .map(job_function_mapping)
    .fillna("Other")
)

In [34]:
encoders_dict = {}

columns_to_encode = ['job_function_group']
for col in columns_to_encode:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders_dict[col] = le

df.head()

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,company_process_time,company_size,company_industry,job_description,salary,job_description_cleaned,annotations,skills,job_function_group
0,1,Facility Maintenance & Smart Warehouse Manager,Bandung,IDR,Manajer/Asisten Manajer,5 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Manufaktur,Pemeliharaan",NaN,NaN,NaN,NaN,Deskripsi PekerjaanRequirements :D3/SI from re...,NaN,deskripsi pekerjaanrequirements si from reputa...,{'text': 'deskripsi pekerjaanrequirements si f...,"['electrical inspection', 'management system',...",7
1,2,Procurement Department Head,Jakarta Raya,IDR,Manajer/Asisten Manajer,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Manufaktur,Pembelian/Manajemen Material",NaN,25 days,51 - 200 pekerja,Manajemen/Konsulting HR,Job Role: 1. Responsible for material availabi...,NaN,job role responsible for material availabili...,{'text': 'job role responsible for material av...,"['heavy equipment', 'contract management', 'pr...",7
2,3,SALES ADMIN,Jakarta Barat,IDR,Supervisor/Koordinator,4 tahun,Sarjana (S1),Penuh Waktu,"Penjualan / Pemasaran,Penjualan Ritel","Waktu regular, Senin - Jumat;Bisnis (contoh: K...",30 days,51 - 200 pekerja,Umum & Grosir,Internal Sales & AdminJob Description :We are ...,NaN,internal sales adminjob description we are loo...,{'text': 'internal sales adminjob description ...,"['microsoft office', 'heat exchanger', 'carbon...",9
3,4,City Operation Lead Shopee Express (Cirebon),Cirebon,IDR,Supervisor/Koordinator,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Pelayanan,Logistik/Rantai Pasokan","Tip;Waktu regular, Senin - Jumat;Kasual (conto...",21 days,2001 - 5000 pekerja,Retail/Merchandise,Job Description:Responsible for HSE implementa...,NaN,job description responsible for hse implementa...,{'text': 'job description responsible for hse ...,"['operation management', 'analytical skill', '...",7
4,5,Japanese Interpreter,Bekasi,IDR,Pegawai (non-manajemen & non-supervisor),2 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Lainnya,Jurnalis/Editor",NaN,23 days,201 - 500 pekerja,Manajemen/Konsulting HR,Overview: Our clients is manufacture for autom...,NaN,overview our clients is manufacture for automo...,{'text': 'overview our clients is manufacture ...,"['japanese', 'translator', 'english', 'non', '...",0


In [35]:
y = df["job_function_group"].fillna("unknown")
print("n_classes:", y.nunique())
print(y.value_counts())

n_classes: 10
job_function_group
9    5424
6    2895
3    1929
7    1638
4    1476
2    1302
5     767
0     714
8     599
1     438
Name: count, dtype: int64


In [36]:
RANDOM_STATE = 42
MAX_PER_CLASS = 600

dfs = []

for label, group in df.groupby("job_function_group"):
    if len(group) > MAX_PER_CLASS:
        group = group.sample(
            n=MAX_PER_CLASS,
            random_state=RANDOM_STATE
        )
    dfs.append(group)

df = pd.concat(dfs).reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5837 entries, 0 to 5836
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       5837 non-null   int64  
 1   job_title                5837 non-null   object 
 2   location                 5837 non-null   object 
 3   salary_currency          5835 non-null   object 
 4   career_level             5837 non-null   object 
 5   experience_level         5837 non-null   object 
 6   education_level          5837 non-null   object 
 7   employment_type          5837 non-null   object 
 8   job_function             5837 non-null   object 
 9   job_benefits             4876 non-null   object 
 10  company_process_time     4531 non-null   object 
 11  company_size             5244 non-null   object 
 12  company_industry         5564 non-null   object 
 13  job_description          5837 non-null   object 
 14  salary                  

In [37]:
df["role_text"] = df["job_description"] + " - " + df['experience_level'] + " - " + df["education_level"]

In [38]:
df.head()

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,company_process_time,company_size,company_industry,job_description,salary,job_description_cleaned,annotations,skills,job_function_group,role_text
0,5894,Photographer,Jakarta Raya,IDR,Pegawai (non-manajemen & non-supervisor),2 tahun,Tidak terspesifikasi,Kontrak,"Seni/Media/Komunikasi,Seni / Desain Kreatif","Waktu regular, Senin - Jumat;Kasual (contoh: K...",24 days,501 - 1000 pekerja,Transportasi/Logistik,What is ShipperShipper is a growing technology...,8000000.0,what is shippershipper is a growing technology...,{'text': 'what is shippershipper is a growing ...,"['warehouse management', 'post production', 's...",0,What is ShipperShipper is a growing technology...
1,16927,HEAD CREATIVE,Jakarta Raya,IDR,Supervisor/Koordinator,2 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Seni/Media/Komunikasi,Seni / Desain Kreatif",Asuransi kesehatan;Bisnis (contoh: Kemeja),25 days,201 - 500 pekerja,Kesehatan/Medis,HEAD CREATIVEKualifikasiMemiliki 2 tahun penga...,NaN,head creativekualifikasimemiliki tahun pengala...,{'text': 'head creativekualifikasimemiliki tah...,"['social medium', 'e commerce', 'target market...",0,HEAD CREATIVEKualifikasiMemiliki 2 tahun penga...
2,2243,Japanese Interpreter,Jakarta Barat,IDR,Pegawai (non-manajemen & non-supervisor),3 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Lainnya,Jurnalis/Editor","Tip;Asuransi kesehatan;Waktu regular, Senin - ...",5 days,51 - 200 pekerja,Manajemen/Konsulting HR,JAPANESE INTERPRETER (TANGERANG) [50020]COMPAN...,NaN,japanese interpreter tangerang company categor...,{'text': 'japanese interpreter tangerang compa...,"['japanese', 'electrical', 'communication', 'e...",0,JAPANESE INTERPRETER (TANGERANG) [50020]COMPAN...
3,15212,Content Creator,Jakarta Barat,IDR,Pegawai (non-manajemen & non-supervisor),3 tahun,Sarjana (S1),Penuh Waktu,"Seni/Media/Komunikasi,Seni / Desain Kreatif","Waktu regular, Senin - Jumat;Bisnis (contoh: K...",24 days,1- 50 pekerja,Makanan & Minuman/Katering/Restoran,CONTENT CREATORLocation : ...,5000000.0,content creatorlocation j...,{'text': 'content creatorlocation jakarta bara...,"['digital marketing', 'social medium', 'adobe ...",0,CONTENT CREATORLocation : ...
4,32356,Graphic Designer,Bali,IDR,Pegawai (non-manajemen & non-supervisor),1 tahun,Sarjana (S1),Penuh Waktu,"Seni/Media/Komunikasi,Seni / Desain Kreatif",NaN,NaN,NaN,Olahraga,Responsibilities :Taking design briefs to unde...,3600000.0,responsibilities taking design briefs to under...,{'text': 'responsibilities taking design brief...,"['design brief', 'graphic design', 'adobe phot...",0,Responsibilities :Taking design briefs to unde...


In [39]:
X_text = df["role_text"].astype(str)
y = df["job_function_group"].astype(int)

num_classes = y.nunique()

In [40]:
X_train, X_test, y_train, y_test = train_test_split(
    X_text,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

data = (X_train, y_train), (X_test, y_test)

# Model for Predict Job Function

### N-gram vectors

In [41]:
NGRAM_RANGE = (1, 2)
TOP_K = 20000
TOKEN_MODE = 'word'
MIN_DOCUMENT_FREQUENCY = 2

def ngram_vectorize(train_texts, train_labels, val_texts):
    kwargs = {
        'ngram_range': NGRAM_RANGE,
        'dtype': 'float32',
        'strip_accents': 'unicode',
        'decode_error': 'replace',
        'analyzer': TOKEN_MODE,
        'min_df': MIN_DOCUMENT_FREQUENCY
    }
    vectorizer = TfidfVectorizer(**kwargs)

    X_train = vectorizer.fit_transform(train_texts)
    X_val = vectorizer.transform(val_texts)

    selector = SelectKBest(f_classif, k=min(TOP_K, X_train.shape[1]))
    selector.fit(X_train, train_labels)
    X_train = selector.transform(X_train)
    X_val = selector.transform(X_val)

    X_train = X_train.toarray()
    X_val = X_val.toarray()
    return X_train, X_val, vectorizer, selector

In [42]:
# X_train, X_test = ngram_vectorize(X_train, y_train, X_test)

### Last Layer

In [43]:
def _get_last_layer_units_and_activation(num_classes):
    if num_classes == 2:
        activation = 'sigmoid'
        units = 1
    else:
        activation = 'softmax'
        units = num_classes
    return units, activation

### Build n-gram model

In [44]:
def mlp_model(num_layers, units, dropout_rate, input_shape, num_classes):
    op_units, op_activation = _get_last_layer_units_and_activation(num_classes)
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dropout(rate=dropout_rate))

    for _ in range(num_layers-1):
        model.add(layers.Dense(units=units, activation='relu'))
        model.add(layers.Dropout(rate=dropout_rate))

    model.add(layers.Dense(units=op_units, activation=op_activation))
    return model

### Train Your Model

In [45]:
(train_texts, train_labels), (val_texts, val_labels) = data
X_train, X_val, vectorizer, selector = ngram_vectorize(train_texts, train_labels, val_texts)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:2043: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. float32 'dtype' will be converted to np.float64.
  warnings.warn(


In [46]:
def train_ngram_model(
    data,
    learning_rate=1e-3,
    epochs=1000,
    batch_size=128,
    num_layers=2,
    units=64,
    dropout_rate=0.2
):

    model = mlp_model(
        num_layers=num_layers,
        units=units,
        dropout_rate=dropout_rate,
        input_shape=X_train.shape[1:],
        num_classes=num_classes
    )

    if num_classes == 2:
        loss = 'binary_crossentropy'
    else:
        loss = 'sparse_categorical_crossentropy'
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])

    callbacks = [tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=2
    )]

    history = model.fit(
        X_train,
        train_labels,
        epochs=epochs,
        callbacks=callbacks,
        validation_data=(X_val, val_labels),
        verbose=2,
        batch_size=batch_size
    )

    history = history.history
    print('Validation accuracy: {acc}, loss: {loss}'.format(
            acc=history['val_acc'][-1], loss=history['val_loss'][-1]))

    model.save('mlp_model.keras')
    return history['val_acc'][-1], history['val_loss'][-1]

In [47]:
train_ngram_model(data, learning_rate=1e-3, epochs=1000, batch_size=128, num_layers=2, units=64, dropout_rate=0.2)

Epoch 1/1000
37/37 - 1s - 34ms/step - acc: 0.5309 - loss: 2.2154 - val_acc: 0.7277 - val_loss: 2.0814
Epoch 2/1000
37/37 - 0s - 12ms/step - acc: 0.7509 - loss: 1.9030 - val_acc: 0.7620 - val_loss: 1.7616
Epoch 3/1000
37/37 - 0s - 12ms/step - acc: 0.7770 - loss: 1.5424 - val_acc: 0.7783 - val_loss: 1.4450
Epoch 4/1000
37/37 - 0s - 11ms/step - acc: 0.8010 - loss: 1.2379 - val_acc: 0.7808 - val_loss: 1.2010
Epoch 5/1000
37/37 - 0s - 12ms/step - acc: 0.8128 - loss: 1.0033 - val_acc: 0.7885 - val_loss: 1.0329
Epoch 6/1000
37/37 - 0s - 11ms/step - acc: 0.8327 - loss: 0.8449 - val_acc: 0.7954 - val_loss: 0.9173
Epoch 7/1000
37/37 - 0s - 11ms/step - acc: 0.8505 - loss: 0.7241 - val_acc: 0.8031 - val_loss: 0.8394
Epoch 8/1000
37/37 - 0s - 11ms/step - acc: 0.8631 - loss: 0.6357 - val_acc: 0.8031 - val_loss: 0.7852
Epoch 9/1000
37/37 - 0s - 11ms/step - acc: 0.8709 - loss: 0.5765 - val_acc: 0.8039 - val_loss: 0.7466
Epoch 10/1000
37/37 - 0s - 11ms/step - acc: 0.8843 - loss: 0.5134 - val_acc: 0.804

(0.801369845867157, 0.6367565393447876)

### Inferensi

In [48]:
model = models.load_model('mlp_model.keras')

def predict_job_function(texts, vectorizer, selector, model):
    X = vectorizer.transform(texts)
    X = selector.transform(X).toarray()
    probs = model.predict(X)
    labels = probs.argmax(axis=1)
    return labels, probs

In [49]:
persona_0_creative_media = ["""Saya memiliki pengalaman panjang di bidang kreatif dan media, dengan fokus utama pada produksi konten visual dan pengelolaan komunikasi digital. 
Selama beberapa tahun terakhir, saya terbiasa bekerja dengan berbagai format konten seperti desain grafis, video pendek, fotografi, serta penulisan 
konten untuk media sosial dan platform digital lainnya. Saya memahami bagaimana sebuah pesan dapat disampaikan secara efektif melalui visual dan 
narasi yang kuat, sehingga audiens dapat terhubung secara emosional dengan brand atau produk yang ditampilkan. Dalam pekerjaan sehari-hari, saya 
sering terlibat dalam proses brainstorming ide kreatif bersama tim, mulai dari tahap konsep hingga eksekusi akhir. Saya terbiasa mengembangkan 
konsep kampanye, menentukan tone komunikasi, serta memastikan konsistensi identitas visual di berbagai kanal. Selain itu, saya juga terbiasa 
bekerja dengan deadline yang ketat dan melakukan revisi berdasarkan masukan dari stakeholder atau klien. Saya memiliki ketertarikan yang besar 
terhadap tren media digital, perkembangan platform sosial, dan perubahan perilaku audiens. Hal ini mendorong saya untuk terus belajar dan 
menyesuaikan gaya konten agar tetap relevan. Saya terbiasa menggunakan berbagai tools desain dan editing, serta memahami dasar analisis performa 
konten untuk melihat efektivitas kampanye yang dijalankan. Bagi saya, dunia kreatif bukan hanya soal estetika, tetapi juga tentang bagaimana sebuah
ide dapat dikomunikasikan dengan jelas dan berdampak. Saya menikmati proses eksplorasi ide, eksperimen visual, dan kolaborasi lintas fungsi dengan 
tim marketing maupun komunikasi. Saya tertarik untuk terus berkembang di bidang creative dan media, khususnya dalam peran yang memungkinkan saya 
berkontribusi pada pengembangan brand dan storytelling yang kuat.
"""]

persona_1_education = ["""
Saya memiliki latar belakang dan pengalaman panjang di bidang pendidikan, baik dalam konteks pengajaran langsung maupun pengembangan materi 
pembelajaran. Saya terbiasa berinteraksi dengan peserta didik dari berbagai latar belakang, memahami kebutuhan belajar mereka, serta menyesuaikan 
metode pengajaran agar materi dapat dipahami dengan lebih efektif. Saya percaya bahwa proses belajar tidak hanya tentang penyampaian materi, tetapi 
juga tentang membangun pemahaman, motivasi, dan rasa ingin tahu. Dalam peran saya sebagai pendidik, saya sering terlibat dalam penyusunan silabus, 
perencanaan pembelajaran, serta evaluasi hasil belajar. Saya terbiasa menggunakan berbagai pendekatan, baik konvensional maupun berbasis teknologi, 
untuk mendukung proses belajar mengajar. Saya juga memiliki pengalaman dalam melakukan pendampingan dan bimbingan kepada peserta didik yang 
membutuhkan perhatian khusus. Saya memiliki ketertarikan pada pengembangan sistem pendidikan, inovasi metode pembelajaran, serta pemanfaatan 
teknologi untuk meningkatkan kualitas pendidikan. Saya percaya bahwa pendidikan memiliki peran penting dalam membentuk karakter dan kompetensi 
seseorang. Oleh karena itu, saya selalu berusaha menciptakan lingkungan belajar yang kondusif, inklusif, dan mendorong partisipasi aktif. Selain 
mengajar, saya juga terbiasa bekerja secara administratif di lingkungan pendidikan, seperti pengelolaan data akademik dan koordinasi kegiatan 
pembelajaran. Saya memiliki kemampuan komunikasi yang baik, sabar, dan terorganisir. Saya tertarik untuk terus berkontribusi di dunia pendidikan,
baik sebagai pengajar, pengembang kurikulum, maupun peran lain yang mendukung proses belajar.
"""]

persona_2_engineering = ["""
Saya memiliki latar belakang di bidang engineering dan pengalaman bekerja dalam lingkungan teknis yang menuntut ketelitian serta pemahaman sistem
yang kuat. Saya terbiasa menganalisis permasalahan teknis, merancang solusi, serta memastikan implementasi berjalan sesuai dengan spesifikasi dan 
standar yang telah ditetapkan. Dalam pekerjaan saya, aspek keselamatan, efisiensi, dan keandalan selalu menjadi prioritas utama. Saya sering bekerja 
dengan dokumentasi teknis, perhitungan, serta koordinasi dengan berbagai pihak seperti tim operasional dan manajemen proyek. Saya memahami bahwa 
sebuah solusi engineering tidak hanya harus bekerja secara teknis, tetapi juga harus dapat diterapkan secara praktis dan berkelanjutan. Oleh karena
itu, saya terbiasa mempertimbangkan berbagai faktor seperti biaya, waktu, dan risiko dalam setiap keputusan teknis. Saya menikmati proses pemecahan
masalah dan tantangan teknis yang kompleks. Saya terbiasa bekerja secara sistematis dan terstruktur, serta mampu bekerja baik secara mandiri maupun 
dalam tim. Saya juga memiliki ketertarikan untuk terus mempelajari teknologi dan metode baru yang dapat meningkatkan kualitas solusi engineering 
yang saya kerjakan. Bagi saya, bidang engineering adalah tentang mengubah konsep dan teori menjadi solusi nyata yang dapat memberikan manfaat. 
Saya tertarik untuk terus berkembang dalam peran engineering yang memungkinkan saya berkontribusi pada pengembangan sistem, infrastruktur, atau
produk yang andal dan efisien.
"""]

persona_3_finance_accounting = ["""
Saya memiliki pengalaman panjang di bidang keuangan dan akuntansi, dengan fokus pada pengelolaan data keuangan yang akurat dan transparan. 
Saya terbiasa menangani pencatatan transaksi, penyusunan laporan keuangan, serta analisis anggaran. Ketelitian dan konsistensi adalah hal yang 
sangat saya jaga dalam bekerja dengan angka dan data finansial. Dalam keseharian, saya sering bekerja dengan laporan keuangan bulanan dan tahunan,
melakukan rekonsiliasi, serta memastikan kepatuhan terhadap standar akuntansi. Saya memahami pentingnya data keuangan sebagai dasar pengambilan 
keputusan bisnis, sehingga saya selalu berusaha menyajikan informasi yang jelas dan dapat dipercaya. Saya juga terbiasa bekerja dengan berbagai 
pihak, termasuk tim internal dan auditor, untuk memastikan proses keuangan berjalan dengan baik. Saya memiliki kemampuan analisis yang kuat dan 
mampu mengidentifikasi potensi risiko maupun peluang dari sisi keuangan. Selain itu, saya terbiasa menggunakan tools seperti spreadsheet dan sistem
akuntansi untuk mendukung pekerjaan. Saya tertarik untuk terus berkembang di bidang finance dan accounting, khususnya dalam peran yang
memungkinkan saya berkontribusi pada stabilitas dan pertumbuhan organisasi melalui pengelolaan keuangan yang baik.
"""]

persona_4_hr_administration = ["""
Saya memiliki pengalaman di bidang human resources dan administrasi, dengan fokus pada pengelolaan sumber daya manusia dan proses administratif 
perusahaan. Saya terbiasa menangani data karyawan, administrasi kepegawaian, serta mendukung proses rekrutmen dan onboarding. Saya memahami
pentingnya ketertiban administrasi dalam mendukung operasional organisasi. Dalam peran saya, saya sering menjadi penghubung antara karyawan dan 
manajemen, memastikan informasi tersampaikan dengan jelas dan tepat. Saya memiliki kemampuan komunikasi yang baik dan terbiasa menangani dokumen 
serta data dengan tingkat kerahasiaan yang tinggi. Saya juga terbiasa bekerja dengan prosedur dan kebijakan perusahaan. Saya tertarik pada 
pengembangan sistem HR yang efisien dan berorientasi pada kesejahteraan karyawan. Saya percaya bahwa pengelolaan sumber daya manusia yang baik 
dapat menciptakan lingkungan kerja yang produktif dan harmonis. Oleh karena itu, saya selalu berusaha bekerja secara rapi, terstruktur, dan
profesional dalam setiap tugas administrasi yang saya tangani.
"""]

persona_5_hospitality_service = ["""
Saya memiliki pengalaman panjang di bidang hospitality dan layanan pelanggan, dengan fokus pada pemberian pelayanan yang ramah dan profesional. 
Saya terbiasa berinteraksi langsung dengan pelanggan, memahami kebutuhan mereka, serta memastikan pengalaman yang positif selama menggunakan 
layanan. Saya memahami bahwa kepuasan pelanggan merupakan faktor utama dalam industri jasa. Saya sering bekerja dalam lingkungan yang dinamis dan 
menuntut kecepatan serta ketepatan. Saya terbiasa menangani keluhan pelanggan dengan sikap tenang dan solutif. Selain itu, saya juga memahami 
pentingnya kerja sama tim dalam memastikan kelancaran operasional layanan. Saya memiliki kemampuan komunikasi interpersonal yang baik dan mampu 
bekerja dalam sistem shift. Saya menikmati pekerjaan yang melibatkan interaksi dengan banyak orang dan memberikan solusi secara langsung. 
Saya tertarik untuk terus berkembang di bidang hospitality dan service, khususnya dalam peran yang menekankan kualitas layanan dan kepuasan
pelanggan.
"""]

persona_6_it_software = ["""
Saya memiliki latar belakang dan pengalaman di bidang teknologi informasi dan pengembangan perangkat lunak. Saya terbiasa bekerja dengan sistem
berbasis software, memahami alur pengembangan aplikasi, serta memecahkan masalah teknis menggunakan pendekatan logis. Saya memiliki ketertarikan
besar terhadap teknologi dan bagaimana teknologi dapat digunakan untuk meningkatkan efisiensi kerja. Saya sering bekerja dengan data, aplikasi,
dan sistem backend, serta memahami konsep database dan integrasi sistem. Saya terbiasa bekerja secara terstruktur dan mengikuti proses pengembangan 
yang sistematis. Selain itu, saya juga mampu bekerja dalam tim dan berkomunikasi dengan pihak non-teknis untuk menjelaskan solusi yang dikembangkan.
Saya tertarik untuk terus belajar dan mengikuti perkembangan teknologi terbaru. Bagi saya, bidang IT dan software adalah ruang yang terus 
berkembang dan menawarkan tantangan intelektual yang menarik. Saya ingin terus berkontribusi dalam pengembangan solusi digital yang bermanfaat
dan scalable.
"""]

persona_7_manufacturing_operations = ["""
Saya memiliki pengalaman di bidang manufaktur dan operasional, dengan fokus pada pengawasan proses produksi dan efisiensi kerja. 
Saya terbiasa bekerja di lingkungan yang terstruktur dengan prosedur yang jelas. Saya memahami pentingnya kualitas produk, keselamatan kerja, dan 
kelancaran operasional dalam proses manufaktur. Saya sering terlibat dalam pemantauan proses produksi, koordinasi dengan tim, serta memastikan
target operasional tercapai. Saya terbiasa bekerja dengan jadwal dan target yang ketat, serta memahami pentingnya kerja sama tim dalam 
lingkungan produksi. Saya tertarik pada peningkatan efisiensi proses dan optimalisasi operasional. Saya percaya bahwa perbaikan kecil yang 
konsisten dapat memberikan dampak besar pada produktivitas. Saya ingin terus berkembang di bidang manufacturing dan operations dengan fokus 
pada proses yang efektif dan berkelanjutan.
"""]

persona_8_other = ["""
Saya memiliki latar belakang pekerjaan yang bersifat umum dan lintas fungsi. Saya terbiasa menangani berbagai tugas yang beragam, mulai dari
administrasi hingga dukungan operasional. Saya fleksibel dan mudah beradaptasi dengan lingkungan kerja yang berubah. Saya memiliki kemampuan
belajar yang cepat dan siap mendukung berbagai kebutuhan organisasi. Saya terbiasa bekerja dengan berbagai tim dan memahami pentingnya komunikasi 
yang jelas. Saya tertarik pada peran yang memungkinkan saya berkontribusi secara luas dan mendukung kelancaran operasional perusahaan.
"""]

persona_9_sales_marketing = ["""
Saya memiliki pengalaman di bidang sales dan marketing, dengan fokus pada pengembangan pasar dan pencapaian target penjualan. Saya terbiasa 
membangun hubungan dengan pelanggan, memahami kebutuhan mereka, serta menawarkan solusi yang sesuai. Saya memiliki kemampuan komunikasi dan 
negosiasi yang baik. Saya sering terlibat dalam perencanaan strategi pemasaran, pelaksanaan kampanye, serta evaluasi hasil penjualan. Saya memahami
pentingnya analisis pasar dan perilaku pelanggan dalam menentukan strategi yang efektif. Saya tertarik untuk terus berkembang di bidang sales dan 
marketing dengan fokus pada pertumbuhan bisnis dan kepuasan pelanggan.
"""]

In [50]:
# persona_0_creative_media = ["""
# Saya merupakan mahasiswa fresh graduate yang memiliki ketertarikan besar di bidang kreatif dan media. Selama masa perkuliahan, saya aktif 
# mempelajari dasar-dasar desain visual, produksi konten digital, dan komunikasi kreatif. Saya terbiasa mengerjakan tugas kuliah yang berkaitan 
# dengan pembuatan konten media sosial, desain poster, presentasi visual, serta penulisan konten sederhana. Saya memiliki minat kuat pada storytelling 
# visual dan bagaimana sebuah pesan dapat disampaikan secara menarik kepada audiens. Saya terbiasa bekerja dengan tools desain dasar dan senang 
# mengikuti tren media digital serta perkembangan platform sosial. Sebagai fresh graduate, saya terbuka untuk belajar, menerima masukan, dan 
# mengembangkan kemampuan kreatif saya melalui pengalaman kerja di industri kreatif dan media.
# """]

# persona_1_education = ["""
# Saya merupakan mahasiswa fresh graduate dengan latar belakang pendidikan yang memiliki minat besar di dunia pendidikan dan pembelajaran. 
# Selama masa studi, saya terbiasa mengikuti kegiatan akademik, diskusi kelompok, serta praktik penyusunan materi pembelajaran. Saya memiliki 
# ketertarikan pada proses belajar mengajar dan bagaimana metode pembelajaran dapat disesuaikan dengan kebutuhan peserta didik. Saya juga memiliki 
# pengalaman membantu teman sebaya dalam kegiatan belajar dan aktif dalam organisasi kampus yang berhubungan dengan pendidikan. Saya percaya bahwa 
# pendidikan memiliki peran penting dalam pengembangan individu. Sebagai fresh graduate, saya ingin terus belajar dan berkontribusi dalam bidang 
# pendidikan, baik sebagai pengajar pemula, asisten pendidikan, maupun peran pendukung lainnya.
# """]

# persona_2_engineering = ["""
# Saya merupakan mahasiswa fresh graduate di bidang engineering dengan pemahaman dasar yang kuat terhadap konsep teknis dan pemecahan masalah. 
# Selama perkuliahan, saya terbiasa mengerjakan tugas dan proyek yang menuntut analisis, perancangan, serta penerapan konsep engineering secara 
# sistematis. Saya memiliki minat pada bagaimana teori dapat diterapkan menjadi solusi nyata. Saya terbiasa bekerja dengan perhitungan, simulasi, 
# dan dokumentasi teknis sederhana. Saya juga pernah terlibat dalam proyek kelompok yang melatih kerja sama tim dan komunikasi teknis. Sebagai fresh 
# graduate, saya siap belajar dari pengalaman lapangan dan mengembangkan kemampuan engineering saya secara bertahap.
# """]

# persona_3_finance_accounting = ["""
# Saya merupakan mahasiswa fresh graduate dengan latar belakang keuangan dan akuntansi. Selama masa kuliah, saya mempelajari dasar-dasar pencatatan 
# keuangan, penyusunan laporan keuangan, serta analisis sederhana terhadap data finansial. Saya terbiasa bekerja dengan angka dan data, serta 
# memahami pentingnya ketelitian dalam pengelolaan keuangan. Saya memiliki pengalaman mengerjakan studi kasus dan tugas kelompok yang berkaitan 
# dengan laporan keuangan dan anggaran. Saya tertarik untuk mengembangkan karier di bidang finance dan accounting dan siap belajar langsung dari 
# praktik kerja di dunia profesional.
# """]

# persona_4_hr_administration = ["""
# Saya merupakan mahasiswa fresh graduate yang memiliki ketertarikan pada bidang human resources dan administrasi. Selama perkuliahan, saya terbiasa 
# mengelola dokumen, data akademik, serta terlibat dalam kegiatan organisasi yang melibatkan administrasi dan koordinasi. Saya memiliki kemampuan 
# komunikasi yang baik dan terbiasa bekerja secara rapi dan terstruktur. Saya tertarik pada proses pengelolaan karyawan, rekrutmen, serta administrasi 
# perusahaan. Sebagai fresh graduate, saya siap belajar mengenai sistem HR dan administrasi secara profesional serta berkontribusi mendukung 
# operasional organisasi.
# """]

# persona_5_hospitality_service = ["""
# Saya merupakan mahasiswa fresh graduate yang memiliki minat besar di bidang hospitality dan layanan. Selama masa kuliah, saya terbiasa berinteraksi 
# dengan banyak orang melalui kegiatan organisasi, kepanitiaan, maupun kerja part-time. Saya memahami pentingnya sikap ramah, komunikasi yang baik, 
# dan pelayanan yang profesional. Saya terbiasa bekerja dalam situasi yang dinamis dan melibatkan kerja tim. Saya tertarik untuk mengembangkan karier 
# di bidang hospitality dan service dengan fokus pada kepuasan pelanggan dan kualitas layanan.
# """]

# persona_6_it_software = ["""
# Saya merupakan mahasiswa fresh graduate di bidang IT dan software dengan pemahaman dasar tentang pemrograman dan sistem informasi. Selama kuliah, 
# saya terbiasa mengerjakan tugas dan proyek yang berkaitan dengan pengembangan aplikasi sederhana, pengolahan data, dan pemecahan masalah teknis. 
# Saya memiliki ketertarikan besar pada teknologi dan senang mempelajari tools serta bahasa pemrograman baru. Saya terbiasa bekerja secara logis dan 
# terstruktur, serta mampu belajar secara mandiri. Sebagai fresh graduate, saya siap mengembangkan kemampuan teknis saya melalui pengalaman kerja 
# dan bimbingan dari tim profesional.
# """]

# persona_7_manufacturing_operations = ["""
# Saya merupakan mahasiswa fresh graduate yang memiliki ketertarikan pada bidang manufaktur dan operasional. Selama perkuliahan, saya mempelajari 
# dasar-dasar proses produksi, manajemen operasional, dan efisiensi kerja. Saya terbiasa mengikuti prosedur dan memahami pentingnya kualitas serta 
# keselamatan kerja. Saya memiliki pengalaman kerja kelompok yang melatih kedisiplinan dan koordinasi tim. Sebagai fresh graduate, saya ingin belajar 
# langsung mengenai proses operasional di lapangan dan berkontribusi dalam peningkatan efisiensi kerja.
# """]

# persona_8_other = ["""
# Saya merupakan mahasiswa fresh graduate dengan latar belakang umum dan fleksibel. Selama masa kuliah, saya terbiasa mengerjakan berbagai jenis 
# tugas akademik dan terlibat dalam kegiatan organisasi kampus. Saya mudah beradaptasi, memiliki kemampuan belajar yang cepat, dan siap membantu 
# berbagai kebutuhan tim. Saya tertarik pada peran entry-level yang memberikan kesempatan untuk belajar lintas fungsi dan mengembangkan pengalaman 
# kerja secara menyeluruh.
# """]

# persona_9_sales_marketing = ["""
# Saya merupakan mahasiswa fresh graduate yang memiliki minat di bidang sales dan marketing. Selama masa kuliah, saya terbiasa melakukan presentasi, 
# diskusi, dan kerja kelompok yang melatih kemampuan komunikasi dan persuasi. Saya tertarik pada strategi pemasaran, perilaku konsumen, dan proses 
# penjualan. Saya juga pernah terlibat dalam kegiatan organisasi atau proyek kampus yang berkaitan dengan promosi dan event. Sebagai fresh graduate, 
# saya siap belajar langsung di dunia kerja dan mengembangkan kemampuan sales serta marketing secara profesional.
# """]

In [51]:
persona_random = ["""
Saya adalah seekor kucing dengan kemampuan untuk mencuri makanan, tidur siang panjang, lapar setiap 5 menit, beraktivitas setiap hari, nyakar jok motor
, bercita-cita untuk bisa menjadi sales. Saya menganggap manusia adalah babu yang perlu melayani saya, menggaruk leher saya ketika gatal. Visi saya adalah
untuk menguasasi negara, menanam sawit di kalimantan dan papua."""]

labels, probs = predict_job_function(persona_9_sales_marketing, vectorizer, selector, model)
print(labels)
print(probs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
[9]
[[0.03305921 0.02249145 0.02422929 0.01151702 0.03786547 0.01380605
  0.00541491 0.01076344 0.02799755 0.8128556 ]]


# Model untuk Get Recommendation based on Semantic Matching

### Embedding

In [77]:
EMBED_MODEL_NAME = "paraphrase-multilingual-MiniLM-L12-v2"
embed_model = SentenceTransformer(EMBED_MODEL_NAME)

def embed_texts(texts, batch_size=64):
    return embed_model.encode(texts, show_progress_bar=True, batch_size=batch_size, convert_to_numpy=True)

role_texts = df["role_text"].tolist()
role_embeddings = embed_texts(role_texts)
print("role_embeddings", role_embeddings.shape)

skills_texts = df["skills"].tolist()
skill_embeddings = embed_texts(skills_texts)
print("skill_embeddings", skill_embeddings.shape)

Batches:   0%|          | 0/92 [00:00<?, ?it/s]

role_embeddings (5837, 384)


Batches:   0%|          | 0/92 [00:00<?, ?it/s]

skill_embeddings (5837, 384)


### Indexing

In [78]:
d = role_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(role_embeddings)
print("index ntotal:", index.ntotal)

faiss.write_index(index, "faiss_role_index.idx")
np.save("role_embeddings.npy", role_embeddings)
np.save("skill_embeddings.npy", skill_embeddings)

index ntotal: 5837


### Create Metadata

In [79]:
metadata = df[[
    "id", "job_title", "job_function", "skills", "job_description"
]].to_dict(orient="records")

In [80]:
with open("job_metadata.bin", "wb") as f:
    pickle.dump(metadata, f)

### Search

In [81]:
def search(query_embedding, k=10):
        D, I = index.search(
            query_embedding, k
        )

        results = []
        for dist, idx in zip(D[0], I[0]):
            job = metadata[idx].copy()
            job["distance"] = float(dist)
            job["similarity"] = float(1 / (1 + dist))  # optional
            results.append(job)

        return results

### Get Top Roles by Text

In [82]:
def get_top_roles_by_text(user_text, k=3):
    emb = embed_texts([user_text])
    results = search(query_embedding=emb, k=k)
    return results

In [84]:
rekomendasi = get_top_roles_by_text(persona_9_sales_marketing[0], k=5)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [85]:
rekomendasi

[{'id': 10919,
  'job_title': 'Sales Industri / Retail (Jakarta, Bandung, Semarang, Surabaya)',
  'job_function': 'Penjualan / Pemasaran,Penjualan Ritel',
  'skills': "['sales', 'r', 'mapping', 'forecast']",
  'job_description': 'Pendidikan minal SMA / SMK (lebih disukai dengan jurusan Kimia / Teknik Kimia / Kimia Industri)Pengalaman di posisi Sales minimal 1 tahun, fresh graduate dipersilahkanLebih diutamakan berpengalaman di bidang perekat / kimia, cat, building material & consumer goodsMemiliki penampilan dan kemampuan presentasi yang baikMampu berkomunikasi dengan baikMemiliki kemampuan menjalin dan bernegosiasi dengan baikMampu mencari & mempertahankan hubungan baik dengan customerJujur dan mau belajarTugas & Tanggung Jawab :Mencari order dari existing customer & new customer, serta menjalankan project customer sampai closedMencari NOO & NCOMembuat permintaan sample atas project customer maupun aplikasi baru di existing customerMelakukan penagihan atas A/R customer yang sudah jatu